# Entity Extraction and Alignment


**Necessary files:**
 - event_df = df\_[event]\_clean.csv file with event dataframes with clean unique tweets


In [1]:
#python libraries
import stanza
import numpy as np
import pandas as pd
import os
import re
from tqdm.notebook import tqdm
import time
from collections import Counter, defaultdict


# pickle functions for quick storage and loading of checkpoint files
import pickle

def pickle_file(file_name, file_to_dump):
    directory_path = os.getcwd() + "/../../../../"
    folder_name = file_name.split('_')[0]
    file_path = directory_path +  fr"Dropbox (CBS)/Master thesis data/Candidate Data/{folder_name}/{file_name}"
    with open(file_path, 'wb') as fp:
        pickle.dump(file_to_dump, fp)

def load_pickle(file_name):
    directory_path = os.getcwd() + "/../../../../"
    folder_name = file_name.split('_')[0]
    #folder_name = re.sub(r'[12]', '', folder_name)
    file_path = directory_path + fr"Dropbox (CBS)/Master thesis data/Candidate Data/{folder_name}/{file_name}"
    with open(file_path, "rb") as input_file:
        return pickle.load(input_file)


# 1. Importing the data

In [2]:
def read_event_df(event_name):
    # reading the clean event dataframes based on the event name
    assert event_name in ['greece','channel','tigray','rohingya','afghanistan']
    
    event_path = fr"Dropbox (CBS)/Master thesis data/Event Dataframes/Clean/df_{event_name}_clean.csv"
    directory_path = os.getcwd() + "/../../../../" + event_path
    event_df = pd.read_csv(directory_path, index_col=0)
    event_df.reset_index(drop=True, inplace=True)
    print(f'loaded {event_df.shape[0]} tweets!')
    return event_df

# pick the df 
event_df = read_event_df('afghanistan')
event_df.head()

loaded 283643 tweets!


,id,source,created_at,lang,author_id,text,retweet_count,reply_count,like_count,quote_count,...,year_month,year_calendar_week,date,refugee,migrant,immigrant,asylum_seeker,other,parsing_corpus,frame_identification_corpus
0,1419447696558108674,Twitter Web App,2021-07-26 00:01:17+00:00,en,50626909,Biden authorizes up to $100M for Afghan refuge...,1,0,0,0,...,2021_7,2021_30,2021-07-26,True,False,False,False,False,Biden authorizes up to $100M for Afghan refugees,biden authorizes afghan refugee
1,1419449166904713220,Twitter for Android,2021-07-26 00:07:08+00:00,en,1372157932943503363,@faheem2430 @USAmbKabul @StateDeputySpox gandu...,0,0,0,0,...,2021_7,2021_30,2021-07-26,True,False,False,False,False,gandu afghani . . . . . maderchood u in refuge...,gandu afghani maderchood refugee camp iran int...
2,1419449634179534850,Twitter for iPhone,2021-07-26 00:08:59+00:00,en,159060632,Hazara refugees urge Australian government to ...,1,0,0,0,...,2021_7,2021_30,2021-07-26,True,False,False,False,False,Hazara refugees urge Australian government to ...,hazara refugee urge australian government help...
3,1419450730063224833,Twitter for Android,2021-07-26 00:13:20+00:00,en,1144443713994678273,"""US has a moral obligation to Afghan allies..c...",0,0,1,0,...,2021_7,2021_30,2021-07-26,False,False,False,False,False,""" US has a moral obligation to Afghan allies ....",moral obligation afghan ally conflict future c...
4,1419453607284510721,TweetDeck,2021-07-26 00:24:46+00:00,en,162114001,Fleeing fighting and hoping to head toward som...,2,1,8,0,...,2021_7,2021_30,2021-07-26,True,False,False,False,False,Fleeing fighting and hoping to head toward som...,fleeing fighting hoping head toward something ...


# 2. Entity extraction: Tag tweets using stanza module to get NER and POS tags in tweets. 
Recommended to run on GPU to speed things up.

*The function will also perform dependency parsing used at the end of the analysis

In [5]:
#
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ needed when running first time ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
#stanza.download("en")
#stanza.install_corenlp()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# loading the pipeline
en_nlp = stanza.Pipeline("en",  
                         tokenize_pretokenized=False,
                         ner_batch_size=4096,
                         processors = "tokenize,pos,lemma,depparse,ner")

2021-09-29 21:56:15 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

2021-09-29 21:56:15 INFO: Use device: cpu
2021-09-29 21:56:15 INFO: Loading: tokenize
2021-09-29 21:56:15 INFO: Loading: pos
2021-09-29 21:56:15 INFO: Loading: lemma
2021-09-29 21:56:15 INFO: Loading: depparse
2021-09-29 21:56:16 INFO: Loading: ner
2021-09-29 21:56:18 INFO: Done loading processors!


Run the stanza pipeline on the parsing corpus for each event dataset

In [9]:
event_name = 'greece'
event_df = read_event_df(event_name)
event_tagged_tweets = [en_nlp(tweet_batch) for tweet_batch in tqdm(list(event_df['parsing_corpus']))]
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)

In [ ]:
event_name = 'channel'
event_df = read_event_df(event_name)
event_tagged_tweets = [en_nlp(tweet_batch) for tweet_batch in tqdm(list(event_df['parsing_corpus']))]
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)

In [ ]:
event_name = 'tigray'
event_df = read_event_df(event_name)
event_tagged_tweets = [en_nlp(tweet_batch) for tweet_batch in tqdm(list(event_df['parsing_corpus']))]
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)

In [8]:
event_name = 'rohingya'
event_df = read_event_df(event_name)
event_tagged_tweets = [en_nlp(tweet_batch) for tweet_batch in tqdm(list(event_df['parsing_corpus']))]
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)

100%|██████████████████████████████████████████████████████████████████████████| 22966/22966 [4:58:17<00:00,  1.28it/s]


In [7]:
event_name = 'afghanistan'
#event_df = read_event_df(event_name)
#event_tagged_tweets = [en_nlp(tweet_batch) for tweet_batch in tqdm(list(event_df['parsing_corpus']))]
#pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)
pickle_file(f'{event_name}_tagged_tweets',event_tagged_tweets)

# 3. Entity alignment
## 3.1. Merging step 1

Pre-requisite files:

    - event_tagged_tweets = python object with tagged tweets
    
The first merging step is an overlap between extraction and alignment. First, we extract the entities from the tagged tweets objects. Next, we process them, remove the duplicates by counting their instances and store them in a data frame. The dataframe is sorted by frequency of the entities.


In [10]:
from nltk import word_tokenize

def get_tweet_tags(tagged_tweets):
    """
    Input: pre-tagged tweets
    Output: list of lists containing named entities
    """
    tweet_entities=[]
    excluded_tags = ['CARDINAL', 'DATE', 'QUANTITY', 'TIME', 'PERCENT', 'MONEY', 'ORDINAL']
    for tweet in tqdm(range(len(tagged_tweets))):
            tweet_ner= [ent.text for sent in tagged_tweets[tweet].sentences for ent in sent.ents if ent.type not in excluded_tags]
            tweet_entities.append(tweet_ner)
    return tweet_entities  

def get_ent_head(entity):
    if len(word_tokenize(entity))>1:
        entity_doc = en_nlp(entity)
        #the root of NP has value 0. Since head is only one and stored in a list, we pick item [0]
        return [word.text for tweet_ent in entity_doc.sentences for word in tweet_ent.words if word.head == 0][0]
    else:
        return entity

def process_entity(entity):
    # re.sub removes all non alpanumeric characters (and lower() lowercases)
    # " ".join and split removes the consecutive whitespaces created by replacing characters
    entity = re.sub('[^0-9a-zA-Z]+', ' ', entity.lower())
    return " ".join(entity.split())

def create_entities_df(tagged_tweets):
    
    event_entities = get_tweet_tags(tagged_tweets)
    
    list_of_entities = list()
    for tweet_entities in tqdm(event_entities):
        list_of_entities.append(tweet_entities)

    entity_list = [process_entity(entity) for tweet in list_of_entities for entity in tweet]
    
    #create the count dictionary which will be converted into a df
    counted_entities = Counter(entity_list)
    ent_df = pd.DataFrame(counted_entities.items(),columns=['entity','freq']).sort_values('freq',ascending=False)
    #filter out entities shorter than 2 characters and frequency less than 5
    ent_df = ent_df[ent_df.freq>=5]
    ent_df['len'] = ent_df.entity.apply(lambda x: len(x))
    ent_df = ent_df[ent_df['len']>=2]
    #find head of the entity
    tqdm.pandas()
    ent_df['head'] = ent_df.entity.progress_apply(get_ent_head)
    ent_df.reset_index(drop=True,inplace=True)
    return ent_df[['entity','freq','head']]

def merging_step1(event_name):
    print(f'loading {event_name} tagged tweets...')
    tagged_tweets = load_pickle(f'{event_name}_tagged_tweets')
    ent_df = create_entities_df(tagged_tweets)
    pickle_file(f'{event_name}_ents',ent_df)
    return ent_df


In [ ]:
ent_df = merging_step1('rohingya')

In [ ]:
ent_df = merging_step1('tigray')

In [ ]:
ent_df = merging_step1('greece')

In [ ]:
ent_df = merging_step1('channel')

In [11]:
ent_df = merging_step1('afghanistan')

loading afghanistan tagged tweets...


  0%|          | 0/283643 [00:00<?, ?it/s]

  0%|          | 0/283643 [00:00<?, ?it/s]

  0%|          | 0/5774 [00:00<?, ?it/s]

## 3.2. Merging step 2: Similarity using Sentence BERT embeddings

### 3.2.1. Encode entities with S-BERT embeddings

Pre-requisite files:
 
    - event_ents = df with event entities after 1st merging step to be encoded


In [13]:
def train_embeddings(event_ents):
    from time import time
    from sentence_transformers import SentenceTransformer
    #sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
    sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')

    bert_corpus = list(event_ents['entity'])

    print(f'there are {len(bert_corpus)} entities to be encoded')
    t0 = time()
    ent_embeddings = sbert_model.encode(bert_corpus)
    print(f'Training embeddings took {time()-t0} seconds')
    return ent_embeddings

In [ ]:

event_ents = load_pickle('greece_ents')

document_embeddings = train_embeddings(event_ents)

pickle_file('greece_embeddings_ents', document_embeddings)

In [ ]:
event_ents = load_pickle('rohingya_ents')

document_embeddings = train_embeddings( event_ents)

pickle_file('rohingya_embeddings_ents', document_embeddings)

In [ ]:
event_ents = load_pickle('tigray_ents')

document_embeddings = train_embeddings(event_ents)

pickle_file('tigray_embeddings_ents', document_embeddings)

In [ ]:
event_ents = load_pickle('channel_ents')

document_embeddings = train_embeddings(event_ents)

pickle_file('channel_embeddings_ents', document_embeddings)

In [14]:
event_ents = load_pickle('afghanistan_ents')

document_embeddings = train_embeddings(event_ents)

pickle_file('afghanistan_embeddings_ents', document_embeddings)

there are 5774 entities to be encoded
Training embeddings took 92.59837603569031 seconds


In [15]:
import winsound

#sound warning after the code is done running
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)

In [3]:
print(load_pickle('afghanistan_embeddings_ents'))

[[-0.08485413  0.19364376  0.02283852 ...  0.12003968  0.02054849
  -0.14343844]
 [-0.04287585  0.24025254  0.02856175 ...  0.1124993   0.02472871
  -0.10030735]
 [-0.04273271  0.5226588   0.03199667 ...  0.20131785 -0.0150766
  -0.11604465]
 ...
 [-0.07053123  0.3559419   0.07664003 ...  0.09561284 -0.01742754
   0.02482462]
 [-0.04873066  0.15382236  0.0189761  ...  0.12828653 -0.14659308
  -0.02422144]
 [-0.07438377 -0.00518416  0.00519211 ... -0.05120416  0.00062359
   0.08227898]]


### 3.2.2 Creating a similarity dataframe
The similarity dataframe enables faster lookup and experimentation with optimal thresholds when aligning entities
Pre-requisite files:

    - [event_name]_ents
    - [event_name]_embeddings_ents

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def create_sim_df(event_name, sim_threshold=0.6):
    """
    1. loads necessary files for the selected event 
    2. calculates pairwise cosine similarity of the entity embeddings 
    3. stores the pairs and their similarity in the similarity dataframe"""
    event_entities = load_pickle(f'{event_name}_ents')
    ent_embeddings = load_pickle(f'{event_name}_embeddings_ents')
    rows_list = []
    
    #calculate pairwise similarities
    sims = cosine_similarity(ent_embeddings)
    
    # check whether a pair similarity is above the given threshold, store in the sim_df if true
    for up_ent_id in tqdm(range(len(event_entities))):
        for low_ent_id in range(len(event_entities)):
            dict1 = {}
            # get input row in dictionary format
            # key = col_name
            if sims[up_ent_id][low_ent_id]>sim_threshold:
                dict1.update({'text': event_entities['entity'][up_ent_id], 
                              'text_to_compare':event_entities['entity'][low_ent_id], 
                              'sim':sims[up_ent_id][low_ent_id]}) 
                rows_list.append(dict1)

    sim_df = pd.DataFrame(rows_list)
    return sim_df

In [ ]:
sim_df = create_sim_df('greece')
pickle_file('greece_sim_df_ents',sim_df)

In [ ]:
sim_df = create_sim_df('rohingya')
pickle_file('rohingya_sim_df_ents',sim_df)

In [ ]:
sim_df = create_sim_df('tigray')
pickle_file('tigray_sim_df_ents',sim_df)

In [ ]:
sim_df = create_sim_df('channel')
pickle_file('channel_sim_df_ents',sim_df)

In [17]:
sim_df = create_sim_df('afghanistan')
pickle_file('afghanistan_sim_df_ents',sim_df)

  0%|          | 0/5774 [00:00<?, ?it/s]

In [ ]:
#sim_df enables lookup of the most similar terms 
sim_df = load_pickle('greece_sim_df_ents')
sim_df[sim_df.text=='greece'].sort_values('sim',ascending=False)[:20]


### 3.2.3. Aligning the entities

Pre-requisite files:

    - sim_df = df with all similar entity pairs)
    - entities = df of all entities (including dissimilar) with frequencies

In [10]:
def flatten(list_of_mixed_types):
    # helper function to convert a list of mixed types (strings and lists) into a list
    flat_list = []
    for element in list_of_mixed_types:
        if isinstance(element,list): 
            flat_list.extend(flatten(element))
        else: 
            flat_list.append(element)
    return flat_list

def align_transitively(what_merged):
    """
    function to transitively merge the entities
    """
    merged_dict = defaultdict(list)
    merged_ent = set()
    
    for key,value in tqdm(what_merged.items()):
        merged_ent.update(what_merged[key])
        for item in value:
            merged_dict[key].extend(what_merged[key])
            if item in what_merged.keys():
                merged_dict[key].extend(what_merged[item])
                merged_dict[key] = list(set(merged_dict[key]))

        #print('-------loop------')
        for ent,merged in what_merged.items():
            for item2 in merged:
                if item2 in what_merged[key] and item2 in what_merged[ent] and key!=ent:
                    merged_dict[key].extend(what_merged[key] + what_merged[ent])
                    merged_dict[key].append(ent)
                    merged_ent.add(ent)
        merged_dict[key] = list(set(merged_dict[key]))  
        #merged_dict[key].pop(merged_dict[key].index(key))
    return merged_dict

def finalize_ents(merged_dict):
    final_ents = defaultdict()
    check_set = set()
    for key,value in merged_dict.items():
        if key not in check_set:
            final_ents[key] = merged_dict[key]
            check_set.update(merged_dict[key])
    
    for key in final_ents.keys():
        if key in final_ents[key]:
            final_ents[key].pop(final_ents[key].index(key))
            
    # add manual terms to the dict so (refugees and asyslum seekers) and (migrants and immigrants) are together
    final_ents['refugees'] = ['refugee','asylum seeker','asylum seekers']
    final_ents['migrants'] = ['migrant','immigrant','immigrants']
    return final_ents


def merging_step2(entities,sim_df,low_threshold=0.8, high_threshold =0.9, transitive=False): 
    """
    inputs:
    entities = entity df with entity frequencies
    sim_df = similarity dataframe with all pairs of entities that are at least [threshold] similar
    low_threshold = determines entity merge in the first merging round 
    high_threshold = determines entity merge in the second and subsequent merging rounds if transitive merge is enabled
    transitive = True if enabled
    returns:
    what_merged = dictionary with core entities as keys and lists of their merged entities as values
    """
    what_merged = defaultdict(list)
    merged_ents = set()

    #merge the dataframes so we have information about frequencies of entities in sim_df
    # use outer join to include the entities that are not similar to any other entity (thus not in sim_df)
    merged_df_ = pd.merge(sim_df,entities[entities.freq>=5], how='outer', left_on='text', right_on='entity')
    merged_df = pd.merge(merged_df_,entities[entities.freq>=5], how='outer', left_on='text_to_compare', right_on='entity')
    
    #blank fields in sim_df's text are the entities that are dissimilar to the rest, pass the value from entity text
    merged_df.entity_x.fillna(merged_df.entity_y, inplace=True) 
    merged_df.freq_x.fillna(merged_df.freq_y, inplace=True) 
    
    merged_df.drop(['text','text_to_compare'],axis=1,inplace=True)

    # and select only the rows above the lower threshold, so we do not have to filter by it in the loop
    merged_df_small = merged_df[merged_df.sim>low_threshold]

    print(f'finding merged entities...')
    # create an object to iterate over unique entities
    unique_merged_df = merged_df.groupby(by=['entity_x'], sort = False, as_index=False).agg({'freq_x':'max'}).sort_values('freq_x',ascending=False)

    def get_merged_entities(entity,transitive = False):
        """
        Store merged entities as list
        """
        lookup_df = pd.DataFrame({'entity_x': [], 'entity_y':[]})

        if entity not in merged_ents:
            if transitive == False:
                lookup_df = merged_df_small[merged_df_small.entity_x==entity]
                merged_ents.update(list(lookup_df['entity_y']))
            else:
                lookup_df = merged_df_small[(merged_df_small.entity_x==entity) & (merged_df_small.sim>high_threshold)]
        
        #clean nans 
        lookup_df = lookup_df[~lookup_df['entity_y'].isnull()]
        return list(lookup_df['entity_y'])
    
    tqdm.pandas() 
    #note that in the first round the transitiveness is False
    unique_merged_df['merged'] =  unique_merged_df.entity_x.progress_apply(get_merged_entities)

    #convert the dataframe into a dictionary
    for entity,merged_list in tqdm(zip(unique_merged_df['entity_x'],unique_merged_df['merged'])):
        what_merged[entity] = merged_list.copy()
        if transitive:
            for merged_ent in merged_list:
                what_merged[entity].append(get_merged_entities(merged_ent,transitive=True))
        what_merged[entity] = list(set(flatten(what_merged[entity]) ) ) 
    
    return what_merged




In [11]:
def run_merging_step2(event_name,entity_type='ents',low_threshold=0.8, high_threshold = 0.9, transitive=True):
    sim_df = load_pickle(f'{event_name}_sim_df_{entity_type}')
    entities = load_pickle(f'{event_name}_{entity_type}')

    what_merged = merging_step2(entities,
                                sim_df,
                                low_threshold=low_threshold, 
                                high_threshold=high_threshold,
                                transitive=transitive)
    
    if transitive:
        what_merged = align_transitively(what_merged)
    
    if event_name == 'channel' or event_name == 'greece':
        what_merged['germany'].pop(what_merged['germany'].index('austria'))
        what_merged['germany'].pop(what_merged['germany'].index('austrian'))
    
    if event_name == 'greece':
        what_merged['russia'].pop(what_merged['russia'].index('ukraine'))
            
    what_merged = finalize_ents(what_merged)
    #pickle_file(f'{event_name}_final_{entity_type}',final_ents)
    pickle_file(f'{event_name}_what_merged',what_merged)
    return  what_merged #final_ents,

In [6]:
merged = load_pickle('afghanistan_sim_df_ents')


In [8]:
merged[merged.text=="afghan"]

,text,text_to_compare,sim
0,afghan,afghan,1.000000
1,afghan,afghanistan,0.879364
2,afghan,taliban,0.706843
3,afghan,afghans,0.908519
4,afghan,kabul,0.722377
...,...,...,...
157,afghan,afghanisthan,0.868022
158,afghan,afghanistan libre,0.673830
159,afghan,the afghan defense forces,0.700342
160,afghan,the prisma afghanistan,0.681075


In [ ]:
merged_dict = run_merging_step2('tigray',low_threshold=0.7,high_threshold = 0.9, transitive=False)

In [ ]:
merged_dict = run_merging_step2('rohingya',low_threshold=0.7,high_threshold = 0.9,transitive=False)

In [14]:
merged_dict = run_merging_step2('greece',low_threshold=0.7, high_threshold = 0.9,transitive=False)

finding merged entities...


  0%|          | 0/2076 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [ ]:
merged_dict = run_merging_step2('channel',low_threshold=0.7,high_threshold=0.85, transitive=False)

In [12]:
merged_dict = run_merging_step2('afghanistan',low_threshold=0.707,high_threshold=0.85, transitive=False)

finding merged entities...


  0%|          | 0/5774 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [13]:
print(list(merged_dict.keys())[:20])

['afghan', 'taliban', 'us', 'biden', 'uk', 'pakistan', 'india', 'trump', 'europe', 'turkey', 'canada', 'priti patel', 'muslim', 'iran', 'airbnb', 'uganda', 'change org uk', 'republicans', 'stephen miller', 'australia']


Check whether the similar pairs from sim_df correspond to entities in merged_dict

In [38]:
merged_dict['republicans']

['democrats',
 'republican party',
 'gop',
 'the republican party',
 'republican',
 'conservatives']

In [39]:
sim_df[sim_df.text=='republicans'].sort_values('sim',ascending=False)[:30]

,text,text_to_compare,sim
1483,republicans,republicans,1.000000
1486,republicans,republican,0.955396
1495,republicans,republican party,0.863772
1490,republicans,the republican party,0.849629
1484,republicans,gop,0.808165
1488,republicans,conservatives,0.723283
1485,republicans,democrats,0.705207
1496,republicans,gop sen,0.694603
1492,republicans,trump gop,0.693915
1489,republicans,conservative,0.650459


### 3.2.4. Find entity frequencies

Pre-requisite files:
    
    - [event_name]_what_merged = dictionary of merged entities
    - event_df = event dataset
    
This part serves solely for illustrative purposes of how much have been the counts of analyzable entities increased thanks to entity alignment.

In [51]:
from collections import defaultdict
from itertools import combinations

def get_frequencies(what_merged, event_df):
    """
    in part 1: get frequencies of each item in what_merged dictionary from the corpus
    
    in part 2: correct frequencies for repeating terms
    """
    #~~~~ part 1 ~~~~
    freq_dict = defaultdict(list)
    event_df['parsing_corpus_lowercased'] = event_df['parsing_corpus'].apply(lambda x: ' '.join(i.lower() for i in x.split()))
    for key in tqdm(what_merged.keys()):
        freq_dict2 = dict()
        c = sum(event_df['parsing_corpus_lowercased'].apply(lambda x: x.count(f' {key} ')))
        freq_dict[key].append(c)
        for item in what_merged[key]:
            c2 = sum(event_df['parsing_corpus_lowercased'].apply(lambda x: x.count(f' {item} ')))
            freq_dict2[item]=c2
        freq_dict[key].append(freq_dict2)
    
    #~~~~ part 2 ~~~~
    
    # Instantiate dict to store individual count for entity and merged count
    freq_counts = defaultdict(dict)
    
    # Iterate over entities
    for ent in freq_dict.keys():
        
        #print(f"Ent: {ent}")
        adj_freq = 0
        
        # Store number of occurences of root entity
        root_freq = freq_dict[ent][0]
        
        # Instantiate empty list where entities are stored that have already been counted
        roots = []
        
        # Integrate main entity into dict of merged entities
        ent_dict = freq_dict[ent][1]
        ent_dict[ent] = freq_dict[ent][0]
        
        # Iterate over merged entities based on their length (in term of tokens) in ascending order
        for merged_ent in sorted(ent_dict.keys(), key = len):
            #print(f"Merged Ent: {merged_ent}")
            
            # Create all possible sub-combinations of entity while keeping the order constant
            # For example: "Greece Turkey Border" yields Greece, Turkey, Border, Greece Turkey, Turkey Border, Greece Turkey Border
            merged_ent_list = merged_ent.split(" ")
            all_slices = [merged_ent_list[s:e] for s, e in combinations(range(len(merged_ent_list)+1), 2)]
            all_slices_strings = [" ".join(l) for l in all_slices]
            
            # Iterate over all entities that have already been counted
            for root in roots:
                
                # Check if any of the substring of current entity has already been counted, if yes don't count it again
                if root in all_slices:
                    #print(f"{merged_ent} was removed to prevent double count. {root} has already been counted")
                    break
            
            # If entity has not been counted, count it and add entity to counted entities
            else:
                roots.append([merged_ent])
                #print(roots)
                adj_freq += ent_dict[merged_ent]
                #print(f"Frequency incremented by {ent_dict[merged_ent]}")
        
        # Update frequency dict
        freq_counts[ent]["Root"] = root_freq
        freq_counts[ent]["Overall"] = adj_freq


    return freq_dict,freq_counts

In [ ]:
event_df = read_event_df(rohingya_url)
what_merged = load_pickle('rohingya_what_merged')

freq_dict,total_freq = get_frequencies(what_merged, event_df)

In [ ]:
event_df = read_event_df(tigray_url)
what_merged = load_pickle('tigray_what_merged')

freq_dict,total_freq = get_frequencies(what_merged, event_df)

In [ ]:
event_df = read_event_df(greece_url)
what_merged = load_pickle('greece_what_merged')

freq_dict,total_freq = get_frequencies(what_merged, event_df)
#freq_dict

In [ ]:
event_df = read_event_df(channel_url)
what_merged = load_pickle('channel_what_merged')

freq_dict,total_freq = get_frequencies(what_merged, event_df)
#freq_dict

In [56]:
event_df = read_event_df('afghanistan')
what_merged = load_pickle('afghanistan_what_merged')

freq_dict,total_freq = get_frequencies(what_merged, event_df)

loaded 283643 tweets!


  0%|          | 0/3365 [00:00<?, ?it/s]

In [53]:
print(list(total_freq.keys())[:30])

['afghan', 'us', 'biden', 'uk', 'pakistan', 'india', 'trump', 'europe', 'turkey', 'canada', 'priti patel', 'muslim', 'iran', 'airbnb', 'uganda', 'change org uk', 'republicans', 'stephen miller', 'australia', 'germany', 'china', 'greece', 'un', 'texas', 'russia', 'nato', 'syrian', 'special immigrant visas', 'france', 'indonesia']


In [ ]:
word= 'brexit'

print(total_freq[word])
print(list(dict(sorted(freq_dict[word][1].items(), key=lambda item: item[1],reverse=True)).keys()))


In [57]:
total_freq


defaultdict(dict,
            {'afghan': {'Root': 176382, 'Overall': 331500},
             'us': {'Root': 32725, 'Overall': 66110},
             'biden': {'Root': 17700, 'Overall': 18680},
             'uk': {'Root': 15200, 'Overall': 20989},
             'pakistan': {'Root': 13841, 'Overall': 16085},
             'india': {'Root': 8406, 'Overall': 11129},
             'trump': {'Root': 8557, 'Overall': 9217},
             'europe': {'Root': 4667, 'Overall': 11738},
             'turkey': {'Root': 4616, 'Overall': 5706},
             'canada': {'Root': 3575, 'Overall': 5384},
             'priti patel': {'Root': 1110, 'Overall': 1972},
             'muslim': {'Root': 6009, 'Overall': 13766},
             'iran': {'Root': 3052, 'Overall': 3607},
             'airbnb': {'Root': 1378, 'Overall': 1423},
             'uganda': {'Root': 2104, 'Overall': 2698},
             'change org uk': {'Root': 0, 'Overall': 0},
             'republicans': {'Root': 2357, 'Overall': 7305},
             's

In [47]:
dict(sorted(total_freq.items(), key=lambda item: item[1]['Overall'],reverse=True))

{'afghan': {'Root': 0, 'Overall': 0},
 'us': {'Root': 0, 'Overall': 0},
 'biden': {'Root': 0, 'Overall': 0},
 'uk': {'Root': 0, 'Overall': 0},
 'pakistan': {'Root': 0, 'Overall': 0},
 'india': {'Root': 0, 'Overall': 0},
 'trump': {'Root': 0, 'Overall': 0},
 'europe': {'Root': 0, 'Overall': 0},
 'turkey': {'Root': 0, 'Overall': 0},
 'canada': {'Root': 0, 'Overall': 0},
 'priti patel': {'Root': 0, 'Overall': 0},
 'muslim': {'Root': 0, 'Overall': 0},
 'iran': {'Root': 0, 'Overall': 0},
 'airbnb': {'Root': 0, 'Overall': 0},
 'uganda': {'Root': 0, 'Overall': 0},
 'change org uk': {'Root': 0, 'Overall': 0},
 'republicans': {'Root': 0, 'Overall': 0},
 'stephen miller': {'Root': 0, 'Overall': 0},
 'australia': {'Root': 0, 'Overall': 0},
 'germany': {'Root': 0, 'Overall': 0},
 'china': {'Root': 0, 'Overall': 0},
 'greece': {'Root': 0, 'Overall': 0},
 'un': {'Root': 0, 'Overall': 0},
 'texas': {'Root': 0, 'Overall': 0},
 'russia': {'Root': 0, 'Overall': 0},
 'nato': {'Root': 0, 'Overall': 0},
 '